# **ETL del Proyecto 1: MLOps**
### Tabla user_items  
#### Importando librerias:

In [1]:
import pandas as pd
import ast

## **Empezando la Extracción**

In [ ]:
ruta_users_items = r"C:\\Users\\Usuario\\Desktop\\Labs\\Proyecto_1\\datasets_originales\\australian_users_items.json"

users_items = [] # Lista que guarda los json leidos

with open(ruta_users_items, encoding="utf-8") as items:
    for linea in items.readlines():
        users_items.append(ast.literal_eval(linea))

for i in range(5):   # Verificamos que el contenido se guardo en la lista
    print(users_items[i])

#### Convertimos la lista en DataFrame y visualizamos:

In [3]:
df_items = pd.DataFrame(users_items)
df_items.head(5)

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


## **Empezamos la transformación**

In [4]:
df_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  object
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.4+ MB


Vemos si hay id's de usuarios repetidos

In [5]:
frecuencia = df_items["user_id"].value_counts()
repetidos = frecuencia[frecuencia > 1]
repetidos

user_id
X03-Suits            3
76561198027488037    3
76561198100326818    3
76561198309337430    3
76561198051777058    3
                    ..
ButteryTyrant        2
barren_cookie        2
76561197964310685    2
76561198055330941    2
potisrest0           2
Name: count, Length: 673, dtype: int64

Observamos que hay id's duplicados en nuestro dataframe:

In [6]:
df_items[df_items["user_id"] == "X03-Suits"]

,user_id,items_count,steam_id,user_url,items
14832,X03-Suits,141,76561198085989695,http://steamcommunity.com/id/X03-Suits,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
14837,X03-Suits,141,76561198085989695,http://steamcommunity.com/id/X03-Suits,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
36239,X03-Suits,141,76561198085989695,http://steamcommunity.com/id/X03-Suits,"[{'item_id': '10', 'item_name': 'Counter-Strik..."


Procedemos a eliminarlos:

In [7]:
df_items = df_items.drop_duplicates(subset=["user_id"], keep="first")

# Vemos si ahora hay mas duplicados
frecuencia = df_items["user_id"].value_counts()
repetidos = frecuencia[frecuencia > 1]
print(repetidos)

Series([], Name: count, dtype: int64)


Buscamos desanidar los items, expandimos las listas:

In [8]:
df_items_expandido = df_items.explode("items")
df_items_expandido.head(5)

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '10', 'item_name': 'Counter-Strike..."
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '20', 'item_name': 'Team Fortress ..."
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '30', 'item_name': 'Day of Defeat'..."
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '40', 'item_name': 'Deathmatch Cla..."
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '50', 'item_name': 'Half-Life: Opp..."


Normalizamos las columnas expandidas, creando un dataframe de la columna 'items'

In [9]:
df_items_desanidado = pd.json_normalize(df_items_expandido["items"])
df_items_desanidado.head(5)

,item_id,item_name,playtime_forever,playtime_2weeks
0,10,Counter-Strike,6.0,0.0
1,20,Team Fortress Classic,0.0,0.0
2,30,Day of Defeat,7.0,0.0
3,40,Deathmatch Classic,0.0,0.0
4,50,Half-Life: Opposing Force,0.0,0.0


Reseteamos los ixdex de ambos dataframes, para evitar errores

In [10]:
df_items_expandido = df_items_expandido.reset_index(drop=True)
df_items_desanidado = df_items_desanidado.reset_index(drop=True)

Concatenamos ambos dataframes para crear uno nuevo

In [11]:
df_items_final = pd.concat([df_items_expandido, df_items_desanidado], axis=1)
df_items_final.head(3)

,user_id,items_count,steam_id,user_url,items,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '10', 'item_name': 'Counter-Strike...",10,Counter-Strike,6.0,0.0
1,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '20', 'item_name': 'Team Fortress ...",20,Team Fortress Classic,0.0,0.0
2,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '30', 'item_name': 'Day of Defeat'...",30,Day of Defeat,7.0,0.0


Eliminamos la columna 'items' ya que es innecesaria

In [12]:
df_items_final = df_items_final.drop("items", axis=1)
df_items_final.head(3)

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6.0,0.0
1,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,20,Team Fortress Classic,0.0,0.0
2,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,30,Day of Defeat,7.0,0.0


Eliminamos las columnas 'user_url', 'item_name' y 'playtime_2weeks'. Porque 'item_name' no la necesitamos ya que tenemos el nombre  
de los juegos en steam_games y las otras dos no tienen uso

In [13]:
a_eliminar2 = ['user_url', 'item_name', 'playtime_2weeks']

df_items_final = df_items_final.drop(a_eliminar2, axis=1)
df_items_final.head(5)

,user_id,items_count,steam_id,item_id,playtime_forever
0,76561197970982479,277,76561197970982479,10,6.0
1,76561197970982479,277,76561197970982479,20,0.0
2,76561197970982479,277,76561197970982479,30,7.0
3,76561197970982479,277,76561197970982479,40,0.0
4,76561197970982479,277,76561197970982479,50,0.0


In [14]:
df_items_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110796 entries, 0 to 5110795
Data columns (total 5 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   items_count       int64  
 2   steam_id          object 
 3   item_id           object 
 4   playtime_forever  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 195.0+ MB


## **Tablas para los EndPoints**
### **Creamos una copia del dataframe de user_items para crear una tabla para la segunda consulta de nuestra API:**  
**def userdata (User_id : str):** 

In [15]:
df_user_data = df_items_final.copy()

Como la columna 'item_id' es de tipo object por los NaN's, la cambiaremos a int, primero convirtiendo  
los NaN's en cero y luego usando astype()

In [16]:
df_user_data['item_id'] = df_user_data['item_id'].fillna(0)

df_user_data["item_id"] = df_user_data["item_id"].astype(int)

Eliminamos columna innecesarias:

In [17]:
drops = ["steam_id", "playtime_forever"]
df_user_data = df_user_data.drop(drops, axis=1)

In [18]:
df_user_data.head(3)

,user_id,item_id
0,76561197970982479,10
1,76561197970982479,20
2,76561197970982479,30


#### **Traemos los csv transformados y los limpiamos de columnas innecesarias**

In [19]:
df_games = pd.read_parquet("./datasets/steam_games.parquet")

df_reviews = pd.read_parquet("./datasets_endpoints/user_reviews_sentiment.parquet")

In [20]:
drops2 = ["genres", "app_name", "release_date", "developer"]
df_games_limpio = df_games.drop(drops2, axis=1)

Hacemos merge() entre los dataframes df_user_data (user_items) y df_games_limpio (steam_games)  
**Esta tabla ya estara lista para exportar**

In [21]:
df_merged = pd.merge(df_user_data, df_games_limpio, on="item_id", how="inner")
df_merged.head(3)

,user_id,item_id,price
0,76561197970982479,10,9.99
1,76561197970982479,20,4.99
2,76561197970982479,30,4.99


### **Creamos una copia del dataframe de user_items para crear una tabla para la tercera consulta de nuestra API:**  
**def UserForGenre( genero : str ):** 

In [24]:
df_user_genre = df_items_final.copy()
df_user_genre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110796 entries, 0 to 5110795
Data columns (total 4 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   steam_id          object 
 2   item_id           object 
 3   playtime_forever  float64
dtypes: float64(1), object(3)
memory usage: 156.0+ MB


Como la columna 'item_id' es de tipo object por los NaN's, la cambiaremos a int, primero convirtiendo  
los NaN's en cero y luego usando astype()

In [25]:
df_user_genre['item_id'] = df_user_genre['item_id'].fillna(0)

df_user_genre["item_id"] = df_user_genre["item_id"].astype(int)

Convertimos la columna 'playtime_forever' a int:

In [27]:
df_user_genre["playtime_forever"] = df_user_genre["playtime_forever"].fillna(0)

df_user_genre["playtime_forever"] = df_user_genre["playtime_forever"].astype(int)

Eliminamos la columna steam_id:

In [29]:
df_user_genre = df_user_genre.drop("steam_id", axis=1)
df_user_genre.head(3)

,user_id,item_id,playtime_forever
0,76561197970982479,10,6
1,76561197970982479,20,0
2,76561197970982479,30,7


## **Empezamos la Carga**
**Guardaremos los dataframes en archivos CSV**

Dataset para EDA

In [15]:
df_items_final.to_parquet("./datasets/user_items.parquet")

Dataset para EndPoint (Consulta 2: def userdata (User_id : str) )

In [31]:
df_merged.to_parquet("./datasets_endpoints/user_data.parquet")

Dataset para EndPoint (Consulta 3: def UserForGenre( genero : str ) )

In [32]:
df_user_genre.to_parquet("./datasets_endpoints/items_user_genre.parquet")